In [1]:
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from xgboost import XGBRFClassifier
import numpy as np
import pandas as pd

In [7]:
df = pd.read_csv("https://projects.fivethirtyeight.com/soccer-api/club/spi_matches.csv").dropna()
df = df[df.league=="Barclays Premier League"].reset_index(drop=True)

In [13]:
import warnings
warnings.filterwarnings("ignore")
df['proj_h_margin'] = df['proj_score1'] - df['proj_score2']
df['act_h_margin'] = df['score1'] - df['score2']

projs = []
acts = []

# The projected result is derived from the projected home margin.
for x in df['proj_h_margin']:
    if x > 0:
        projs.append("PROJECTED HOME")
    elif x < 0:
        projs.append("PROJECTED AWAY")
    else:
        projs.append("PROJECTED DRAW")

# Likewise, the actual result is derived from the actual home margin.
for y in df['act_h_margin']:
    if y > 0:
        acts.append("HOME")
    elif y < 0:
        acts.append("AWAY")
    else:
        acts.append("DRAW")

# We don't need these columns for the construction of the model.
df = df.drop(columns=['proj_h_margin', 'act_h_margin'])

df['proj_result'] = projs
df['result'] = acts

matches = []

# This is where the rubber hits the road, or where we compare the projected results vs. the actual results.
# If they match, it's "MATCH." If they don't, it's "NO MATCH."
for z in range(len(df)):
    p = df['proj_result'][z]
    a = df['result'][z]
    if p == "PROJECTED HOME" and a == "HOME":
        matches.append("MATCH")
    elif p == "PROJECTED HOME" and a == "AWAY":
        matches.append("NO MATCH")
    elif p == "PROJECTED HOME" and a == "DRAW":
        matches.append("NO MATCH")
    elif p == "PROJECTED AWAY" and a == "AWAY":
        matches.append("MATCH")
    elif p == "PROJECTED AWAY" and a == "HOME":
        matches.append("NO MATCH")
    elif p == "PROJECTED AWAY" and a == "DRAW":
        matches.append("NO MATCH")
    elif p == "PROJECTED DRAW" and a == "HOME":
        matches.append("NO MATCH")
    elif p == "PROJECTED DRAW" and a == "AWAY":
        matches.append("NO MATCH")
    else:
        matches.append("MATCH")

df['match'] = matches

# Defining the predictor and targets.
X_final = df[['proj_score1','proj_score2','score1','score2']]
y_final = df['match']

a_score = []

def model():
  # This is the heavy lifting part of the experiment, where we generate 1,000 rounds of predictions and the random_state increases by 1 each time.
  # After each iteration, the accuracy value is appended to a list where we will examine the min, mean, and max values.
    count = 0
    while count < 1000:
        X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.2, random_state=count)
        model = XGBRFClassifier(max_depth=6, objective='binary:logistic', eval_metric='logloss', n_estimators=400).fit(X_train, y_train)
        predictions = model.predict(X_test)
        accuracy = accuracy_score(y_test, predictions)
        a_score.append(accuracy)
        count += 1

model = model()

# Accuracy results based on 1,000 rounds of y_test and prediction comparisons.
print("Accuracy Values: Minimum: {} | Mean: {} | Max: {}".format(np.round(np.min(a_score),3), np.round(np.mean(a_score),3), np.round(np.max(a_score),3)))

Accuracy Values: Minimum: 0.941 | Mean: 0.968 | Max: 0.997
